In [ ]:
import json
import pandas as pd
import urllib
import time
import re
from bs4 import BeautifulSoup

base_url = "https://www.zwiftpower.com/api3.php?do=analysis_list&zwift_id="

# top riders JSON is situated here ↓
# https://www.zwiftpower.com/cache3/lists/2_standings_.json .
# Unomment the lines below to use this list of top ~1000 riders
#with urllib.request.urlopen("https://www.zwiftpower.com/"\
#                            "cache3/lists/2_standings_.json") as response:
#    raw_json_top = response.read()
    
# all riders JSON is located here ↓
# https://www.zwiftpower.com/cache3/global/rider_list.json
# A version of the list from 2021-03-17 with most of the JSON attributes
# stripped (only inlcuding name, zwift id, ftp and rank) is available in
# the github repo and can be used by uncommenting the next lines
with open("minified_rider_list.json") as file:
    raw_json_top = file.read()

json_top_rankings_zp = json.loads(raw_json_top)

# We are interested in the the top zwifters personal data (name and ID)
top_zwifter = json_top_rankings_zp.get("data")
top_zwids, riders = [], []

# make lists of zwift_ids and rider names
for zwifter in top_zwifter:
    
    # we only want to check riders with zwiftpower rank less than 500
    if int(zwifter.get("rank")[:-3]) < 500:
        top_zwids.append(zwifter.get("zwid"))
        riders.append(zwifter.get("name"))
    
zwift_ids, setids, names, dates, titles, device_pm, device_st, \
power300_pm, power300_st, group  = [], [], [], [], [], [], [], [], [], []

progress = 0
time_request = 0

print(f"Selected {len(top_zwids)} Zwifters with rank below 500 from"\
      f" {len(top_zwifter)} Zwifters total.")

# iterate over all riders in top rankings
for zwifter, rider in zip(top_zwids, riders):
    
    # print progress
    print(f"# {progress:4}. ID: {zwifter:8}."\
          f" {progress/len(top_zwids) * 100:5.2f} % finished."\
          f" Request time: {time_request:.2f} s.")
    progress += 1
    
    # wait for some seconds – don't flood the server
    time.sleep(0.5)
    
    time_request_start = time.time()
    # request analysis JSON for rider
    with urllib.request.urlopen(base_url + str(zwifter)) as response:
        request = response.read()
    json_str = json.loads(request)
    time_request_end = time.time()
    time_request = time_request_end - time_request_start

    # get every race for current rider
    races = json_str.get("data")
          
    
    # iterate over each race
    for race in races:
        
        # if the race has exactly 2 power values for 300s power...
        # ... and none of the values are zero
        if race.get("power300") and 0 not in race.get("power300"):
            num_devices = len(race.get("power300"))
        else:
            num_devices = 0
        if num_devices == 2:
            devices = []
            
            # device names are stored as name1, name2 etc.
            for i in range(num_devices):
                devices.append(race.get("name" + str(i+1)))
            
            if 0 in devices:
                continue

            
            # regex matching kickr but not if bike or snap is included in name
            wahoo_search = [i for i in devices if \
                            (re.search(".*kickr*", i, re.I) and not \
                             re.search(".*bike.*|.snap.*", i, re.I))]
            
            # if match for kickr and 2 devices and max 10 % difference between
            # device power readings, add values to lists.
            if wahoo_search and num_devices == 2 and \
                abs(race.get("power300")[0]/race.get("power300")[1] - 1) \
                < 0.1:
                wahoo_index = devices.index(wahoo_search[0])
                setids.append(race.get("set_id"))
                zwift_ids.append(zwifter)
                names.append(rider)
                dates.append(race.get("date"))
                titles.append(race.get("title"))
                group.append("wahoo")
                if wahoo_index == 1:
                    device_pm.append(devices[0])
                    device_st.append(devices[1])
                    power300_pm.append(race.get("power300")[0])
                    power300_st.append(race.get("power300")[1])
                else:
                    device_pm.append(devices[1])
                    device_st.append(devices[0])
                    power300_pm.append(race.get("power300")[1])
                    power300_st.append(race.get("power300")[0])
                                                   
            
            # regex matching neo but not if 2 comes after neo (except when the)
            # 2 is followed by a 0 (as in 20), so that a Tacx Neo 2017 is 
            # included in the match, but not Tacx Neo 2T
            neo_search = [i for i in devices if \
                          (re.search(".*neo(?![2t])", i, re.I) and not\
                           re.search("2(?!0)", i, re.I))]
            
            if neo_search and num_devices == 2 and \
                abs(race.get("power300")[0]/race.get("power300")[1] - 1) \
                < 0.1:
                neo_index = devices.index(neo_search[0])
                setids.append(race.get("set_id"))
                zwift_ids.append(zwifter)
                names.append(rider)
                dates.append(race.get("date"))
                titles.append(race.get("title"))
                group.append("neo")
                if neo_index == 1:
                    device_pm.append(devices[0])
                    device_st.append(devices[1])
                    power300_pm.append(race.get("power300")[0])
                    power300_st.append(race.get("power300")[1])
                else:
                    device_pm.append(devices[1])
                    device_st.append(devices[0])
                    power300_pm.append(race.get("power300")[1])
                    power300_st.append(race.get("power300")[0])

print(f"Finished! Saved {len(zwift_ids)} rows.")

Selected 39264 Zwifters with rank below 500 from 300205 Zwifters total.
#    0. ID:        8.  0.00 % finished. Request time: 0.00 s.
#    1. ID:       49.  0.00 % finished. Request time: 1.12 s.
#    2. ID:       77.  0.01 % finished. Request time: 1.00 s.
#    3. ID:      108.  0.01 % finished. Request time: 1.02 s.
#    4. ID:      118.  0.01 % finished. Request time: 0.90 s.
#    5. ID:      119.  0.01 % finished. Request time: 0.76 s.
#    6. ID:      161.  0.02 % finished. Request time: 0.80 s.
#    7. ID:      167.  0.02 % finished. Request time: 1.09 s.
#    8. ID:      170.  0.02 % finished. Request time: 0.96 s.
#    9. ID:      197.  0.02 % finished. Request time: 1.00 s.
#   10. ID:      199.  0.03 % finished. Request time: 1.16 s.
#   11. ID:      204.  0.03 % finished. Request time: 0.83 s.
#   12. ID:      225.  0.03 % finished. Request time: 0.99 s.
#   13. ID:      246.  0.03 % finished. Request time: 0.87 s.
#   14. ID:      250.  0.04 % finished. Request time: 0.86 s

In [ ]:
top_zwifter

In [71]:
# mash all of the lists into a much more efficient pandas dataframe structure
df = pd.DataFrame({"zwift_id": zwift_ids, "name": names, "date": dates, \
                   "group": group, "title": titles, "pm": device_pm,\
                   "st": device_st, "p300s_pm": power300_pm,\
                   "p300s_st": power300_st}, index=setids)

# calculate the delta (diffrence) between power meter and smart trainer
df["delta"] = df["p300s_pm"] - df["p300s_st"]

In [72]:
# print key stats (mean + std.dev) for the Kickr and Neo
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
print(f"Wahoo Kickr statistics:")
print(df.loc[df["group"] == "wahoo"].describe())
print(f"\n\n Tacx Neo statistics:")
print(df.loc[df["group"] == "neo"].describe())

Wahoo Kickr statistics:
           zwift_id     p300s_pm     p300s_st        delta
count  9.258000e+03  9258.000000  9258.000000  9258.000000
mean   5.316325e+05   374.778354   373.887989     0.890365
std    5.707533e+05    52.105218    52.070517    10.143604
min    1.653000e+03    98.000000    99.000000   -41.000000
25%    1.134010e+05   347.000000   346.000000    -5.000000
50%    3.491540e+05   377.000000   377.000000     1.000000
75%    7.557130e+05   408.000000   407.000000     7.000000
max    3.105026e+06   529.000000   540.000000    44.000000


 Tacx Neo statistics:
           zwift_id    p300s_pm    p300s_st       delta
count  7.220000e+02  722.000000  722.000000  722.000000
mean   3.382669e+05  367.759003  364.055402    3.703601
std    3.836611e+05   47.817732   47.268815   11.114495
min    1.260800e+04  152.000000  154.000000  -35.000000
25%    4.412500e+04  343.000000  339.000000   -3.000000
50%    1.829720e+05  369.000000  366.000000    5.000000
75%    5.049500e+05  398.7500

In [73]:
df.to_csv("data.csv")

In [ ]:
# URL Test string: url = "https://www.zwiftpower.com/api3.php?do=analysis_list&zwift_id=136809"
# sample JSON: json_string = '{"data":[{"set_id":"6525","date":"1576609826","title":"Zwift - WORLDOFFUSION Racing Series (A)","p":0,"name1":"Wahoo Kickr","power5":[1069],"power15":[943],"power60":[623],"power300":[420],"power1200":[356],"missing":[19],"zero":[300]},{"set_id":"6547","date":"1576608978","title":"Zwift - WORLDOFFUSION Racing Series (A)","p":0,"name1":"Wahoo Kickr","power5":[1069,1099],"power15":[943,1025],"power60":[623,639],"power300":[420,427],"power1200":[356,358],"missing":[19,33],"zero":[228,190],"name2":"Quarq DZero PM"},{"set_id":"22930","date":"1586284211","title":"Zwift - 3R Spring Loaded Race Series - Round 2of6 (A) - 3rd I think...","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1193,1125],"power15":[1119,1068],"power60":[617,606],"power300":[410,411],"power1200":[362,368],"missing":[1,113],"zero":[91,40],"name2":"Wahoo Kickr to Bolt"},{"set_id":"27167","date":"1587617818","title":"Zwift - Ramp Test - dual power test","p":0,"name1":"Wahoo Kickr 18 ","power5":[501,518],"power15":[500,510],"power60":[499,502],"power300":[459,462],"power1200":[310,314],"missing":[362,0],"zero":[368,268],"name2":"Quarq DZero PM"},{"set_id":"27322","date":"1587657768","title":"Zwift - OH CRIT!! - The Double Race - Race 2\/2 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1196,1185],"power15":[1075,1051],"power60":[602,583],"power300":[424,408],"power1200":[358,346],"missing":[0,0],"zero":[28,14],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"28374","date":"1587837299","title":"Crit City Slam \/\/ Zwift Classic","p":0,"name1":"Quarq DZero PM to Zwift","power5":[984,941],"power15":[943,908],"power60":[690,657],"power300":[456,433],"power1200":[390,371],"missing":[4,135],"zero":[66,55],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"28460","date":"1588006764","title":"Zwift - DBR Wahoo League (A) - 3rd","p":0,"name1":"Quarq DZero PM to Zwift","power5":[954,943],"power15":[918,889],"power60":[670,659],"power300":[416,405],"power1200":[348,335],"missing":[33,0],"zero":[164,175],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"28975","date":"1588153318","title":"Zwift - USA Cycling Race League by Indoor Specialist (A) - 1st :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1045,1030],"power15":[994,963],"power60":[692,660],"power300":[440,425],"power1200":[387,370],"missing":[16,0],"zero":[129,142],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"29336","date":"1588223698","title":"Zwift - Christopher MCGLINCHEY [VIT] \'s Meetup - Three Sisters","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1222,1233],"power15":[1043,1072],"power60":[609,602],"power300":[403,401],"power1200":[283,292],"missing":[0,38],"zero":[421,262],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"30406","date":"1588408787","title":"Zwift - Cycling Ireland Zwift League Rd 6 Road to Ruins (A) - 1st :) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1247,1220],"power15":[1111,1119],"power60":[567,577],"power300":[448,456],"power1200":[378,388],"missing":[22,28],"zero":[126,114],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"30638","date":"1588434441","title":"Zwift - YANA Pro\/AM Invitiational (A) -2nd :)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[785,800],"power15":[749,772],"power60":[576,597],"power300":[473,478],"power1200":[379,387],"missing":[0,2],"zero":[122,98],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"31873","date":"1588703060","title":"3R Spring Loaded Race Series - Round 6of6","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1211,1179],"power15":[1118,1087],"power60":[621,584],"power300":[481,468],"power1200":[394,374],"missing":[21,102],"zero":[108,154],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"32664","date":"1588876697","title":"Zwift - WTRL Team Time Trial (C)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[685,698],"power15":[635,636],"power60":[531,513],"power300":[437,423],"power1200":[385,370],"missing":[0,227],"zero":[66,390],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"32907","date":"1589013400","title":"Zwift - CIZL Rd 7 \u2013 Sean Lynch Memorial (A)2nd- 1st CIZL :) ","p":0,"name1":"Quarq DZero PM to Zwift","power5":[988,995],"power15":[948,937],"power60":[727,720],"power300":[468,451],"power1200":[383,373],"missing":[17,490],"zero":[10,17],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"33785","date":"1589259553","title":"Zwift - Christopher MCGLINCHEY [VIT] \'s Meetup - Three Sisters - ZADA Test","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1127,1173,1152],"power15":[1001,1062,1043],"power60":[554,561,553],"power300":[446,463,446],"power1200":[383,393,384],"missing":[1,57,210],"zero":[130,84,158],"name2":"Quarq DZero PM to Bolt ANT+","name3":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"34041","date":"1589308290","title":"3R & Wahoo Ride Real Race Series - Round 1 of 7","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1144,1143],"power15":[969,953],"power60":[608,604],"power300":[428,425],"power1200":[370,363],"missing":[28,149],"zero":[0,2],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"34240","date":"1589362888","title":"Zwift - USA Cycling Race League by Indoor Specialist (A) - 5th","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1275,1274],"power15":[1184,1174],"power60":[608,616],"power300":[445,442],"power1200":[346,341],"missing":[13,0],"zero":[24,54],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"34739","date":"1589475973","title":"Zwift - WTRL Team Time Trial Premier League (C) - 2nd","p":0,"name1":"Quarq DZero PM to Bolt ANT+","power5":[674,666],"power15":[623,617],"power60":[550,548],"power300":[442,440],"power1200":[406,403],"missing":[12,666],"zero":[7,12],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"35239","date":"1589618488","title":"Zwift - Cycling Ireland Zwift League Rd 8 (A) - 1st :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1190,1201],"power15":[1075,1064],"power60":[627,626],"power300":[455,468],"power1200":[411,414],"missing":[0,4],"zero":[124,193],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"36110","date":"1589912613","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 2 of 7 (A) 2nd - BRUTAL! New 20min power PB ","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1187,1190],"power15":[1110,1118],"power60":[649,643],"power300":[468,471],"power1200":[426,430],"missing":[1,59],"zero":[106,160],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"36682","date":"1590084032","title":"Zwift - Tour for All Invitational Race \/\/ Men (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[841,826],"power15":[771,768],"power60":[612,616],"power300":[505,506],"power1200":[419,419],"missing":[0,226],"zero":[5,14],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"36951","date":"1590195903","title":"Mens Tour of the Gila - Stage 1 - WTRL Team Time Trial","p":0,"name1":"Quarq DZero PM to Zwift","power5":[580,581],"power15":[562,553],"power60":[537,536],"power300":[455,457],"power1200":[421,418],"missing":[0,184],"zero":[6,3],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"37175","date":"1590248775","title":"Zwift - MEN\'S TOUR OF THE GILA STAGE 2 CIRCUIT RACE (A) - 2nd","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1209,1226],"power15":[1101,1072],"power60":[590,595],"power300":[481,477],"power1200":[384,378],"missing":[0,28],"zero":[60,133],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"37427","date":"1590335222","title":"Zwift - MEN\'S TOUR OF THE GILA STAGE 3 EPIC KOM QUEEN STAGE (A) - 2nd, 3rd GC & KOM comp :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1062,1064],"power15":[839,845],"power60":[594,600],"power300":[459,462],"power1200":[425,424],"missing":[0,25],"zero":[59,106],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"37852","date":"1590518058","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 3 of 7 (A) Forgot TT bike for the TT.... new 20 min power though haha! ","p":0,"name1":"Quarq DZero PM to Zwift","power5":[605,609],"power15":[585,579],"power60":[516,515],"power300":[469,468],"power1200":[433,430],"missing":[21,16],"zero":[0,2],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"38348","date":"1590689320","title":"Zwift - Tour for All Invitational Race \/\/ Men (A) - 3rd","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1243,1254],"power15":[1132,1126],"power60":[609,617],"power300":[468,468],"power1200":[370,367],"missing":[0,8],"zero":[29,56],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"38647","date":"1590829001","title":"Zwift - Cycling Ireland Zwift League Rd 10 (A) - 1st :) overall wrapped up! :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1213,1204],"power15":[1150,1139],"power60":[563,548],"power300":[488,489],"power1200":[346,333],"missing":[11,20],"zero":[85,176],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"39117","date":"1591121106","title":"Zwift - UNO X CUP powered by DCU & Feltet.dk (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1024,1019],"power15":[965,955],"power60":[654,662],"power300":[440,441],"power1200":[359,358],"missing":[8,0],"zero":[53,84],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"40547","date":"1591725630","title":"Zwift - UNO X CUP powered by DCU & Feltet.dk (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1268,1248],"power15":[1110,1088],"power60":[664,672],"power300":[421,421],"power1200":[361,356],"missing":[0,1],"zero":[100,179],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"41360","date":"1592038662","title":"Zwift - Cycling Ireland Zwift League Rd 12 (A) - 1st :) Thanks CI for a great series ","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1339,1308],"power15":[1228,1213],"power60":[633,613],"power300":[446,437],"power1200":[330,324],"missing":[44,5],"zero":[340,553],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"42010","date":"1592332608","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 5 of 7 (A) - 2nd","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1132,1108],"power15":[1024,1013],"power60":[694,701],"power300":[397,384],"power1200":[330,321],"missing":[5,0],"zero":[93,154],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"42661","date":"1592608711","title":"Zwift - VIRTUAL JOE MARTIN STAGE RACE ITT","p":0,"name1":"Quarq DZero PM to Zwift","power5":[624,619],"power15":[559,546],"power60":[512,523],"power300":[463,473],"power1200":[0,0],"missing":[14,171],"zero":[0,0],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"42865","date":"1592668429","title":"Zwift - VIRTUAL JOE MARTIN STAGE RACE - 3rd","p":0,"name1":"Quarq DZero PM to Zwift","power5":[987,951],"power15":[925,895],"power60":[584,561],"power300":[450,433],"power1200":[342,340],"missing":[0,184],"zero":[411,1046],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"42981","date":"1592754307","title":"Zwift - VIRTUAL JOE MARTIN STAGE RACE (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[900,912],"power15":[842,847],"power60":[670,676],"power300":[436,430],"power1200":[353,349],"missing":[36,125],"zero":[447,1073],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"43247","date":"1592937407","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 6 of 7 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[708,717],"power15":[672,671],"power60":[574,573],"power300":[478,476],"power1200":[328,320],"missing":[24,58],"zero":[369,427],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"43248","date":"1592937407","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 6 of 7 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[708,717],"power15":[672,671],"power60":[574,573],"power300":[478,476],"power1200":[328,320],"missing":[24,58],"zero":[176,236],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"44270","date":"1593542038","title":"Zwift - 3R & Wahoo Ride Real Race Series - Round 7 of 7 (A) boom :) 1st win in a while!","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1055,1059],"power15":[1011,1006],"power60":[693,692],"power300":[431,432],"power1200":[370,370],"missing":[43,4],"zero":[16,2],"name2":"Wahoo Kickr to Bolt"},{"set_id":"46468","date":"1594835354","title":"Zwift - USA Cycling Race League by Indoor Specialist (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1079,1092],"power15":[1019,1023],"power60":[666,663],"power300":[425,424],"power1200":[331,327],"missing":[13,12],"zero":[21,18],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"49948","date":"1597393928","title":"Zwift - Off the MAAP Invitational Race \/\/ Stage 1 (Men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1133,1091],"power15":[1050,992],"power60":[560,544],"power300":[423,410],"power1200":[367,360],"missing":[4,0],"zero":[262,210],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"50863","date":"1597912878","title":"Zwift - Off the MAAP Invitational Race \/\/ Stage 3 (men) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[784,793],"power15":[734,724],"power60":[590,591],"power300":[422,424],"power1200":[361,365],"missing":[20,0],"zero":[269,161],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"51698","date":"1598378231","title":"Zwift - RHYS HOWELL [CANYON]\'s Meetup - Big Foot Hills","p":0,"name1":"Quarq DZero PM to Zwift","power5":[752,764],"power15":[739,748],"power60":[569,560],"power300":[437,427],"power1200":[360,350],"missing":[2,6],"zero":[67,110],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"58195","date":"1601370141","title":"Zwift - KISS at Race - Americas AM (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1142,1143],"power15":[1071,1061],"power60":[616,632],"power300":[396,383],"power1200":[341,327],"missing":[23,25],"zero":[170,306],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"61279","date":"1602527485","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[924,928],"power15":[857,869],"power60":[677,680],"power300":[444,452],"power1200":[346,357],"missing":[17,0],"zero":[369,612],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"61280","date":"1602527485","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[924,928],"power15":[857,869],"power60":[677,680],"power300":[444,452],"power1200":[346,357],"missing":[17,0],"zero":[263,143],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"63076","date":"1603129532","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[709,780],"power15":[597,748],"power60":[501,501],"power300":[421,427],"power1200":[389,393],"missing":[42,0],"zero":[148,963],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"63087","date":"1603129532","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) ","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[0,0],"power15":[0,0],"power60":[0,0],"power300":[0,0],"power1200":[0,0],"missing":[0,0],"zero":[0,0],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"65173","date":"1603741473","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[830,828],"power15":[817,808],"power60":[670,664],"power300":[422,428],"power1200":[353,359],"missing":[22,0],"zero":[107,74],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"67265","date":"1604342926","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[665,624],"power15":[567,575],"power60":[499,506],"power300":[427,435],"power1200":[397,404],"missing":[11,0],"zero":[68,15],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"69411","date":"1604951019","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1082,1124],"power15":[1042,1072],"power60":[614,614],"power300":[468,470],"power1200":[329,336],"missing":[19,0],"zero":[425,254],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"70215","date":"1605114833","title":"Zwift - Ramp Test - Turbo Test","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[367,393],"power15":[364,388],"power60":[359,384],"power300":[319,342],"power1200":[0,0],"missing":[176,174],"zero":[0,0],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"70216","date":"1605116746","title":"Zwift - Ramp Test - turbo test","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[385,406],"power15":[381,402],"power60":[379,399],"power300":[339,357],"power1200":[0,0],"missing":[11,0],"zero":[36,40],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"72030","date":"1605552386","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[736,727],"power15":[557,583],"power60":[490,500],"power300":[426,433],"power1200":[397,405],"missing":[35,0],"zero":[48,18],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"74687","date":"1606160644","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1089,1092],"power15":[1022,1033],"power60":[659,674],"power300":[473,482],"power1200":[351,354],"missing":[17,38],"zero":[174,78],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"76774","date":"1606557602","title":"Zwift - Cycling Ireland Zwift League Rd 3 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1227,1194],"power15":[1156,1111],"power60":[629,638],"power300":[440,445],"power1200":[310,304],"missing":[13,0],"zero":[85,182],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"77669","date":"1606762138","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[697,735],"power15":[664,667],"power60":[552,559],"power300":[432,436],"power1200":[412,416],"missing":[9,90],"zero":[61,509],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"78873","date":"1607000954","title":"Zwift - Watopia - UCI Tacx NEO 2T","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[1125,1148],"power15":[983,1036],"power60":[564,589],"power300":[324,340],"power1200":[254,267],"missing":[0,0],"zero":[20,11],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"79245","date":"1607065467","title":"Zwift - Watopia - Tacx Neo 2T","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[1039,1056],"power15":[937,968],"power60":[505,520],"power300":[337,348],"power1200":[255,264],"missing":[0,7],"zero":[3,8],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"79669","date":"1607162103","title":"Zwift - Cycling Ireland Zwift League Rd 4 (A)","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[1134,1217],"power15":[1026,1087],"power60":[549,580],"power300":[374,391],"power1200":[295,306],"missing":[23,0],"zero":[149,105],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"80358","date":"1607324355","title":"Zwift - Watopia","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[1017,1044],"power15":[619,637],"power60":[361,361],"power300":[277,288],"power1200":[248,258],"missing":[0,0],"zero":[6,3],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"80800","date":"1607451486","title":"Zwift - Watopia","p":0,"name1":"Tacx Neo 2T - Zwift","power5":[1177,1208],"power15":[907,941],"power60":[481,493],"power300":[252,258],"power1200":[225,234],"missing":[0,0],"zero":[48,29],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"82216","date":"1607767148","title":"Zwift - Cycling Ireland Zwift League Rd 5 (A) - 1st","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1026,1027],"power15":[776,788],"power60":[526,538],"power300":[461,469],"power1200":[405,392],"missing":[10,0],"zero":[128,206],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"83146","date":"1607968448","title":"Zwift - 3R Bologna Race - 2 Laps (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[654,614],"power15":[585,595],"power60":[535,543],"power300":[456,464],"power1200":[364,359],"missing":[0,14],"zero":[234,410],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"84323","date":"1608224371","title":"Zwift - OH MY CRIT! - The Double Race 2\/2 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1243,1252],"power15":[1138,1120],"power60":[606,571],"power300":[395,364],"power1200":[327,315],"missing":[0,0],"zero":[118,225],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"85863","date":"1608578276","title":"Zwift - Team TFC Mad Monday Winter Series (A) - 1st :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1154,1134],"power15":[1061,1060],"power60":[614,594],"power300":[432,421],"power1200":[401,389],"missing":[3,0],"zero":[128,234],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"86145","date":"1608656915","title":"Zwift - 3R Volcano KOM After Party Race - 2 laps + climb (40.2km\/25mi 286m) (A) - 1st :) ","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1129,1158],"power15":[1041,1065],"power60":[604,606],"power300":[436,439],"power1200":[394,396],"missing":[7,0],"zero":[5,18],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"86361","date":"1608706108","title":"Zwift - Christopher McGlinchey\'s Meetup - Three Sisters - ZADA Test","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[999,987],"power15":[957,944],"power60":[599,581],"power300":[444,439],"power1200":[377,372],"missing":[0,0],"zero":[315,212],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"87474","date":"1608995430","title":"Zwift - OH MY CRIT - Saturday Smash Race (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1076,1070],"power15":[994,1000],"power60":[614,594],"power300":[408,408],"power1200":[311,301],"missing":[2,0],"zero":[245,368],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"87695","date":"1609048716","title":"Zwift - KISS 100 Race Series - EU (A) - 1st :)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1177,1177],"power15":[1096,1068],"power60":[490,488],"power300":[408,412],"power1200":[361,358],"missing":[0,0],"zero":[672,1072],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"88489","date":"1609239514","title":"Zwift - KISS Race Series - Americas AM (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1186,1134],"power15":[1057,1015],"power60":[622,639],"power300":[373,380],"power1200":[334,328],"missing":[69,0],"zero":[124,244],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"89100","date":"1609350607","title":"Zwift - MIDWEEK Winter Series by SZR (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1106,1108],"power15":[1062,1045],"power60":[657,653],"power300":[392,399],"power1200":[344,339],"missing":[128,150],"zero":[147,239],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"89520","date":"1609430566","title":"Zwift - OH MY CRIT! - The Double Race 1\/2 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[896,912],"power15":[866,881],"power60":[682,694],"power300":[443,432],"power1200":[348,343],"missing":[0,464],"zero":[66,98],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"89543","date":"1609434106","title":"Zwift - OH MY CRIT! - The Double Race 2\/2 (A)","p":0,"name1":"Quarq DZero PM to Zwift","power5":[1103,1111],"power15":[1058,1058],"power60":[607,605],"power300":[382,382],"power1200":[332,333],"missing":[0,0],"zero":[1,6],"name2":"Wahoo Kickr 18 to Wahoo Bolt"},{"set_id":"89701","date":"1609484676","title":"Zwift - 3R Innsbruckring Race - 33.2km (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[988,954],"power15":[950,921],"power60":[637,634],"power300":[398,390],"power1200":[325,322],"missing":[0,0],"zero":[178,83],"name2":"Quarq DZero PM to Bolt ANT+"},{"set_id":"91395","date":"1609790129","title":"Zwift - Emily\'s Short Mix","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1140,1151],"power15":[1081,1091],"power60":[542,546],"power300":[360,363],"power1200":[325,329],"missing":[0,0],"zero":[52,45],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"91788","date":"1609871307","title":"Zwift - TDZ Stage 1: Standard Ride","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1257,1171],"power15":[726,741],"power60":[453,458],"power300":[327,332],"power1200":[292,294],"missing":[0,5],"zero":[31,11],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"92313","date":"1609955621","title":"Zwift - MIDWEEK Winter Series by SZR (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[922,914],"power15":[906,894],"power60":[723,729],"power300":[397,399],"power1200":[333,337],"missing":[30,0],"zero":[44,18],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"92777","date":"1610038479","title":"Zwift - OH MY CRIT! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1075,1102],"power15":[1028,1029],"power60":[689,696],"power300":[404,408],"power1200":[341,347],"missing":[5,0],"zero":[153,85],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"93221","date":"1610088760","title":"Zwift - TDZ Stage 1: Longer Ride","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[766,795],"power15":[655,652],"power60":[446,426],"power300":[319,324],"power1200":[280,278],"missing":[0,0],"zero":[73,19],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"93679","date":"1610182817","title":"Zwift - KRT\/KISS 80 km Endurance Ride (B)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[642,613],"power15":[533,536],"power60":[391,388],"power300":[300,291],"power1200":[269,270],"missing":[0,0],"zero":[19,7],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"93696","date":"1610186044","title":"Zwift - Cycling Ireland Zwift League Rd 7 (A) - 1st :)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1019,1016],"power15":[980,981],"power60":[650,652],"power300":[372,376],"power1200":[313,319],"missing":[15,0],"zero":[261,163],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"94981","date":"1610390794","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1059,1110],"power15":[1026,1042],"power60":[665,671],"power300":[378,380],"power1200":[339,343],"missing":[0,0],"zero":[80,46],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"96869","date":"1610640785","title":"Zwift - OH MY CRIT! - The Double Race 1\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1116,1125],"power15":[1016,1037],"power60":[655,680],"power300":[425,419],"power1200":[336,338],"missing":[61,0],"zero":[94,77],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"96891","date":"1610643163","title":"Zwift - OH MY CRIT! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1182,1186],"power15":[1082,1086],"power60":[607,599],"power300":[382,385],"power1200":[323,326],"missing":[15,0],"zero":[44,13],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"97704","date":"1610778356","title":"Zwift - XRS\u00a0Races - The Saturday Classic (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1047,1045],"power15":[884,900],"power60":[530,533],"power300":[432,432],"power1200":[388,389],"missing":[0,0],"zero":[259,207],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"99188","date":"1610997487","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[629,607],"power15":[581,582],"power60":[532,530],"power300":[417,413],"power1200":[397,396],"missing":[5,0],"zero":[17,0],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"100697","date":"1611226597","title":"Zwift - KISS Race Series - Americas AM (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1124,1100],"power15":[1045,1038],"power60":[639,637],"power300":[399,392],"power1200":[329,324],"missing":[0,0],"zero":[235,119],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"102890","date":"1611600322","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[901,909],"power15":[864,876],"power60":[571,575],"power300":[438,438],"power1200":[406,405],"missing":[3,0],"zero":[90,48],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"106888","date":"1612208571","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[981,971],"power15":[911,926],"power60":[675,681],"power300":[405,405],"power1200":[344,345],"missing":[7,0],"zero":[378,277],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"110252","date":"1612677352","title":"Zwift - KISS 100 Race Series - EU (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1172,1181],"power15":[1122,1119],"power60":[602,593],"power300":[440,439],"power1200":[353,356],"missing":[18,6],"zero":[890,528],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"111021","date":"1612810528","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[720,708],"power15":[615,617],"power60":[533,537],"power300":[438,440],"power1200":[404,408],"missing":[29,4],"zero":[75,40],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"112857","date":"1613062877","title":"Zwift - OH MY CRIT!! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[958,981],"power15":[939,944],"power60":[802,807],"power300":[417,418],"power1200":[342,342],"missing":[0,0],"zero":[112,48],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"113279","date":"1613112675","title":"Zwift - 3R Innsbruckring Race - 33.2km (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1233,1243],"power15":[1071,1113],"power60":[614,615],"power300":[372,375],"power1200":[293,298],"missing":[1,0],"zero":[145,67],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"115353","date":"1613418454","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[970,944],"power15":[938,930],"power60":[666,678],"power300":[449,450],"power1200":[330,332],"missing":[20,0],"zero":[351,262],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"117187","date":"1613667522","title":"Zwift - OH MY CRIT!! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1102,1085],"power15":[1072,1026],"power60":[631,621],"power300":[433,437],"power1200":[334,336],"missing":[0,0],"zero":[43,132],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"119100","date":"1614020104","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[903,876],"power15":[859,848],"power60":[738,741],"power300":[406,406],"power1200":[354,356],"missing":[9,0],"zero":[50,20],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"120705","date":"1614272475","title":"Zwift - OH MY CRIT!! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1119,1115],"power15":[1059,1063],"power60":[620,630],"power300":[397,402],"power1200":[361,364],"missing":[0,0],"zero":[14,4],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"121393","date":"1614407193","title":"Zwift - XRS\u00a0Races - The Saturday Classic (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[0,0],"power15":[0,0],"power60":[0,0],"power300":[0,0],"power1200":[0,0],"missing":[0,0],"zero":[0,0],"name2":"2021-02-27-062633-ELEMNT BOLT 6D8D-197-0"},{"set_id":"121394","date":"1614407193","title":"Zwift - XRS\u00a0Races - The Saturday Classic (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[752,763],"power15":[726,738],"power60":[522,532],"power300":[432,433],"power1200":[356,358],"missing":[4,78],"zero":[652,651],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"122571","date":"1614628911","title":"Zwift - Premier Division \/\/ Zwift Racing League (men) (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[622,582],"power15":[571,547],"power60":[531,515],"power300":[454,447],"power1200":[403,400],"missing":[24,0],"zero":[22,5],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"123962","date":"1614874076","title":"OH MY CRIT!! - The Double Race 1\/2","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1147,1140],"power15":[1074,1075],"power60":[605,628],"power300":[416,413],"power1200":[319,322],"missing":[23,0],"zero":[143,260],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"123986","date":"1614876578","title":"Zwift - OH MY CRIT!! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1134,1134],"power15":[1062,1062],"power60":[620,619],"power300":[403,405],"power1200":[338,340],"missing":[0,0],"zero":[43,33],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"127001","date":"1615481178","title":"Zwift - OH MY CRIT!! - The Double Race 2\/2 (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1188,1162],"power15":[1066,1066],"power60":[577,565],"power300":[408,410],"power1200":[342,349],"missing":[0,111],"zero":[109,68],"name2":"Quarq AXS DZero to Bolt"},{"set_id":"129129","date":"1615919196","title":"Zwift - CeramicSpeed Race Series: Open Race (A)","p":0,"name1":"Wahoo Kickr 18 to Zwift ANT+","power5":[1007,998],"power15":[900,909],"power60":[566,582],"power300":[435,438],"power1200":[369,376],"missing":[23,146],"zero":[230,259],"name2":"Quarq AXS DZero to Bolt"}]}'